# Sleep stages classification pipeline
___

This notebooks aims to construct our feature matrix from our sleep records [sleep-edf](https://physionet.org/content/sleep-edfx/1.0.0/) from _physionet_. 

We will reuse the chosen features from our exploration notebook.

Since all of the dataset cannot be loaded in memory at the same time, we will have to implement a pipeline, where each step can then be run with only one recording at a time. At the end of this notebook, we will be able to concatenate all resulting features in a single matrix.

In [ ]:
%matplotlib inline

from multiprocessing import Pool, cpu_count

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.model_selection import (train_test_split, KFold)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, confusion_matrix, classification_report, f1_score)

import mne
from mne.time_frequency import psd_welch
from scipy.stats import (skew, kurtosis)
from scipy.signal import butter

from utils import fetch_data

We define a few constants.

In [ ]:
NB_EPOCHS_AWAKE_MORNING = 60
    
SAMPLING_FREQ = 100
NYQUIST_FREQ = SAMPLING_FREQ/2
EPOCH_DURATION = 30. # in seconds
MAX_TIME = EPOCH_DURATION - 1. / SAMPLING_FREQ  # tmax in included

EEG_CHANNELS = [
    'EEG Fpz-Cz',
    'EEG Pz-Oz'
]

# Sleep stages values
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# EEG sub bands labels
DELTA = "delta"
THETA = "theta"
ALPHA = "alpha"
SIGMA = "sigma"
BETA = "beta"

ANNOTATIONS_EVENT_ID = {
    'Sleep stage W': W,
    'Sleep stage 1': N1,
    'Sleep stage 2': N2,
    'Sleep stage 3': N3,
    'Sleep stage 4': N3,
    'Sleep stage R': REM
}

EVENT_ID = {
    "W": W,
    "N1": N1,
    "N2": N2,
    "N3": N3,
    "REM": REM
}

FREQ_BANDS_RANGE = {
    DELTA: [0.5, 4.5],
    THETA: [4.5, 8.5],
    ALPHA: [8.5, 11.5],
    SIGMA: [11.5, 15.5],
    BETA: [15.5, 30]
}

FREQ_BANDS_ORDERS = {
    DELTA: 5,
    THETA: 8,
    ALPHA: 9,
    SIGMA: 9,
    BETA: 14
}

## Preprocessing
___

Our dataset consists of 39 recordings, each containing about 20 hours of EEG, EOG, EMG, and other signals, sampled at 100 or 1Hz.

### 1. Loading recordings informations
___

This file contains information about when a recording was started, at which time the subject went to bed and the amount of sleep he got.

In [ ]:
df_records = pd.read_csv("data/recordings-info.csv")
df_records.head(5)



### 2. Retrieve data from file and filter to only one channel of EEG
___

This step includes:
- Retrieving edf file
- Excluding channels that are not EEG signals
- Retrieving dataframe recordings information

In [ ]:
def fetch_signal(psg_file_name, hypno_file_name):
    """
    returns: mne.Raw of the whole night recording
    """
    raw_data = mne.io.read_raw_edf(psg_file_name, preload=True, stim_channel=None, verbose=False)
    annot = mne.read_annotations(hypno_file_name)
    raw_data.set_annotations(annot, emit_warning=False)
    
    return raw_data

def drop_other_channels(raw_data, channel_to_keep):
    """
    returns: mne.Raw with the two EEG channels and the signal
        between the time the subject closed the lights and the time
        at which the subject woke up
    """
    raw_data.drop_channels([ch for ch in raw_data.info['ch_names'] if ch != channel_to_keep])
    
    return raw_data

def get_recording_info(file_name):
    # Each file is named according to this pattern:
    #   SC4ssNEO-PSG.edf where SS is subject index (3:5) and N is subject night index (5:6)
    subject_index = int(file_name[3:5])
    subject_night = int(file_name[5:6])
    
    return df_records[(df_records['subject'] == subject_index) & (df_records['night'] == subject_night)]

In [ ]:
get_recording_info("SC4121E0-PSG.edf")

### 3. Convert Raw signal to epochs or matrices
___

In [ ]:
def convert_to_epochs(raw_data, info):
    """
    returns:
        mne.Epochs, where the epochs are only choosen if the subject was in bed.
        y: np array of shape (nb_epochs,), which contains each epoch label.
    """

    # Number of seconds since file began
    closed_lights_time = info['LightsOffSecond'].values[0]
    woke_up_time = closed_lights_time + info['NightDuration'].values[0] + NB_EPOCHS_AWAKE_MORNING*EPOCH_DURATION

    raw_data.crop(tmin=closed_lights_time, tmax=min(woke_up_time, raw_data.times[-1]))
    
    events, annot_event_id = mne.events_from_annotations(
        raw_data,
        event_id=ANNOTATIONS_EVENT_ID,
        chunk_duration=EPOCH_DURATION,
        verbose=False)
    
    # Few files do not have N3 sleep (i.e. SC4202EC-Hypnogram), so we have to filter out key-value pairs that are not in the annotations.
    event_id = { 
        event_key: EVENT_ID[event_key] 
        for event_key in EVENT_ID
        if EVENT_ID[event_key] in annot_event_id.values()
    }
    
    epochs = mne.Epochs(
        raw=raw_data,
        events=events,
        event_id=event_id,
        tmin=0.,
        tmax=MAX_TIME,
        preload=True,
        baseline=None,
        verbose=False)
    
    y = np.array([event[-1] for event in epochs.events])
    
    return epochs, y 


def convert_to_matrices(data):
    """
    data: mne.Epochs with only one EEG channel
    
    returns
        - X: Matrix of input values, of size (nb_epochs, sampling_rate*epoch_length=3000)
        - y: Vector of observation labels, of size (nb_epochs,)
    """
    df = data.to_data_frame(picks="eeg", long_format=True)
    df = df.drop(columns=['ch_type', 'channel'])
    df = df.sort_values(by=['epoch', 'time'])
    
    y = df[['epoch', 'condition']].drop_duplicates(keep="first")['condition'].to_numpy()
    X = np.matrix(
        [df[df['epoch'] == epoch]['observation'].to_numpy() for epoch in df['epoch'].unique()]
    )

    return X, y

Complete function to apply in order to preprocess our data:

In [ ]:
def preprocess(data, current_channel, df_info, convert_to_matrix=False):
    """
    data: mne.Raw 
        Instance of all of the night recording and all channels
    current_channel: str 
        Current EEG channel
        
    returns
        - X: Matrix of input values, of size (nb_epochs, sampling_rate*epoch_length=3000)
        - y: Vector of observation labels, of size (nb_epochs,)
    """
    data = drop_other_channels(data, current_channel)
    data, y = convert_to_epochs(data, df_info)
    
    if not convert_to_matrix:
        return data, y
    
    return convert_to_matrices(data)


## Feature extraction
___


```
        Frequency domain
        features                +----> Average delta band +-+
                +-------+       |                           |
      +-------->+  FFT  +-------+      ...                  |
      |         +-------+       |                           |
      |                         +----> Mean frequency  +----+
      |                                                     |
      |                                                     |
      |                                                     |
  X +-+                                                     +-> X'
Input |                                                     |   Features
mne.  |                         +-----> Variance +----------+   np.array
Epochs|                         |                           |   shape:
      |         +----------+    |                           |    (nb_epochs, nb_features)
      +-------->+ get_data +-------------> Mean +-----------+
                +----------+    |                           |
                                |       ...                 |
                                |                           |
                                +-----> Zero cross rate ----+
                                
                                
                                
        Time
        domain features

```

We first have to define a transformer that will extract the values out of an epoch instance.

In [ ]:
def get_data_from_epochs(epochs):
    """
    epochs: mne.Epochs
    
    returns np array of shape (nb_epochs, sampling_rate*epoch_length)
    """
    return epochs.get_data().squeeze()

get_data_from_epochs_transformer = FunctionTransformer(get_data_from_epochs, validate=False)

We then define a skeleton fonction which receives a fonction that is called for every epochs. 

In [ ]:
def get_transformer(get_feature):
    
    def get_one_feature_per_epoch(X, get_feature):
        """
        X: Input matrix (nb_epochs, sampling_rate*epoch_length)
        get_feature: callable 
            generates one feature for each epoch

        returns matrix (nb_epoch,1)
        """
        return [[get_feature(epoch)] for epoch in X]

    return lambda X: get_one_feature_per_epoch(X, get_feature)

def get_transformer_list(get_features):
    
    def get_feature_list_per_epoch(X, get_features):
        """
        X: Input matrix (nb_epochs, sampling_rate*epoch_length)
        get_feature: callable 
            generates a list of features for each epoch

        returns matrix (nb_epoch,nb_features)
        """
        return [get_features(epoch) for epoch in X]

    return lambda X: get_feature_list_per_epoch(X, get_features)

#### 1. Time domain features
___

##### a) Standard statistics
____

We extract features on the distribution of the time domain values of each epoch.

In [ ]:
mean_transformer = FunctionTransformer(get_transformer(np.mean), validate=True)
std_transformer = FunctionTransformer(get_transformer(np.std), validate=True)
skew_transformer = FunctionTransformer(get_transformer(skew), validate=True)
kurtosis_transformer = FunctionTransformer(get_transformer(kurtosis), validate=True)

##### b) Mean crossing rate
____

In [ ]:
def get_zero_crossing_rate(signal):
    """
    Multiplies signal by itself shifted by one.
    If the signal crosses the horizontal axis, the sign will be negative and vice-versa.
    
    Returns nb of time the signal crossed the horizontal axis
    """
    return ((signal[:-1] * signal[1:]) < 0).sum()

def get_mean_crossing_rate(signal):
    return get_zero_crossing_rate(signal - np.mean(signal))

mean_crossing_rate_transformer = FunctionTransformer(get_transformer(get_mean_crossing_rate), validate=True)

##### c) Hjorth parameters
____

In [ ]:
# Code taken from PyEEG: https://github.com/forrestbao/pyeeg

def hjorth(X):
    """ Compute Hjorth mobility and complexity of a time series from either two
    cases below:
        1. X, the time series of type list (default)
        2. D, a first order differential sequence of X (if D is provided,
           recommended to speed up)
    In case 1, D is computed using Numpy's Difference function.
    Notes
    -----
    To speed up, it is recommended to compute D before calling this function
    because D may also be used by other functions whereas computing it here
    again will slow down.
    Parameters
    ----------
    X
        list
        a time series
    D
        list
        first order differential sequence of a time series
    Returns
    -------
    As indicated in return line (mobility, complexity)
    """
    D = np.diff(X)
    D = D.tolist()

    D.insert(0, X[0])  # pad the first difference
    D = np.array(D)

    n = len(X)

    M2 = float(sum(D ** 2)) / n
    TP = sum(np.array(X) ** 2)
    M4 = 0
    for i in range(1, len(D)):
        M4 += (D[i] - D[i - 1]) ** 2
    M4 = M4 / n

    # Hjorth Mobility and Complexity
    mobility = np.sqrt(M2 / TP)
    complexity = np.sqrt(
        float(M4) * TP / M2 / M2
    )
    return [mobility, complexity] # np.concatenate([mobility, complexity], axis=1)

hjorth_transformer = FunctionTransformer(get_transformer_list(hjorth), validate=True)

##### Merging all time domain features with `FeatureUnion`
___

In [ ]:
time_domain_feature_union = FeatureUnion([
    ('mean', mean_transformer),
    ('std', std_transformer),
    ('skew', skew_transformer),
    ('kurtosis', kurtosis_transformer),
    ('mean-crossing-rate', mean_crossing_rate_transformer),
    ('hjorth', hjorth_transformer)
], n_jobs=1)

time_domain_pipeline = Pipeline([
    ('epochs_to_data', get_data_from_epochs_transformer),
    ('time_domain_features', time_domain_feature_union)
])

#### 2. Frequency domain features
___

In [ ]:
def eeg_power_band(epochs):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5]
        Transformed data.
    """
    psds, freqs = psd_welch(epochs, fmin=0.5, fmax=30.)
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS_RANGE.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

eeg_power_bands_transformer = FunctionTransformer(eeg_power_band, validate=False)

In [ ]:
frequency_domain_feature_union = FeatureUnion([
    ('power_band', eeg_power_bands_transformer)
], n_jobs=1)
# frequency_domain_pipeline = make_pipeline()

#### 3. Sub-band features
___

Certain features discriminate more sleep stages when they are calculated only on a particular sub-band. We will provide functions which calculates the value on each sub-band, and then divide the signal before calling these functions.

##### a) Mean energy
___

Mean energy corresponds to $ME = \frac{1}{N}\sum_{t=0}^{N} x_t^2$, where N is the epoch length.

In [ ]:
def get_signal_mean_energy(signal):
    """
    signal: array of (nb_sample_per_epoch,)
    """
    return np.sum(signal**2)*1e6

mean_energy_transformer = FunctionTransformer(get_transformer(get_signal_mean_energy), validate=True)

We will apply 5 IIR filters on all of our epochs in order to calculate time domain features on sub-band epochs.

In [ ]:
def get_pipeline_per_subband(subband_name: str):
    """
    Constructs a pipeline to extract the specified subband related features.
    Output:
        sklearn.pipeline.Pipeline object containing all steps to calculate time-domain feature on the specified subband.
    """
    
    freq_range = FREQ_BANDS_RANGE[subband_name]
    order = FREQ_BANDS_ORDERS[subband_name]
    
    assert len(freq_range) == 2, "Frequency range must only have 2 elements: [lower bound frequency, upper bound frequency]"
    
    bounds = [freq/NYQUIST_FREQ for freq in freq_range]
    b, a = butter(order, bounds, btype='bandpass')
    
    def filter_epochs_in_specified_subband(epochs):
        return epochs.copy().filter(
            l_freq=bounds[0],
            h_freq=bounds[1],
            method='iir',
            n_jobs=1,
            iir_params = {
                'a': a,
                'b': b
            }, verbose=False)
        
    return Pipeline([
        ('filter', FunctionTransformer(filter_epochs_in_specified_subband, validate=False)),
        ('get-values', get_data_from_epochs_transformer),
        ('mean-energy', mean_energy_transformer),
#         ('standard-scaler', StandardScaler()) # Because we scale our features within mean_energy fct, we currently don't have to apply the scaler.
    ])

In [ ]:
subband_feature_union = FeatureUnion([(
        f"{band_name}-filter",
        get_pipeline_per_subband(band_name)
    ) for band_name in FREQ_BANDS_ORDERS.keys()], n_jobs=1)


#### 3. Complete feature extraction pipeline
___

In [ ]:
feature_union = FeatureUnion([
    ('time_domain', time_domain_pipeline),
    ('frequency_domain', frequency_domain_feature_union),
    ('subband_time_domain', subband_feature_union)
], n_jobs=1)

## Extraction
___

In [ ]:
SUBJECTS = range(6)
NIGHT_RECORDINGS = [1, 2]

subject_file_names = fetch_data(subjects=SUBJECTS, recording=NIGHT_RECORDINGS)

psg_file_names = [names[0] for names in subject_file_names]
stage_file_names = [names[1] for names in subject_file_names]

In [ ]:
%%time

def get_features(recording_index):
    """
    recording_index: index starting at 0..nb_files-1.
        ** It does not corresponds to the file indexes if we don't include the first files in the subjects range. **
    Returns features X in a vector of (nb_epochs, nb_features)
    """
    try:
        print("Calculating for file #", recording_index)
        df_info = get_recording_info(psg_file_names[recording_index][-16:])
        raw_data = fetch_signal(psg_file_names[recording_index], stage_file_names[recording_index])
        features_file = []

        for channel in EEG_CHANNELS:
            X_file_channel, y_file_channel = preprocess(raw_data.copy(), channel, df_info)
            X_features = feature_union.fit_transform(X_file_channel)
            features_file.append(X_features)

            print(f"Done extracting {X_features.shape[1]} features on {X_features.shape[0]} epochs for {channel} for file {psg_file_names[recording_index][-16:]}\n")
            assert X_features.shape[0] == len(y_file_channel), "Features and labels must have the same number of epochs"
    
    except Exception as e:
        print(e)
        print("ERROR: for file ", psg_file_names[recording_index])
        raise e

    # Only returns y one time, because both channels refer to the same epochs
    return np.hstack(tuple(features_file)), y_file_channel

with Pool(processes=cpu_count()) as pool:
    observations = pool.map(get_features, range(len(psg_file_names)))
    X, y = zip(*observations)

In [ ]:
X = np.vstack(X)
y = np.hstack(y)
print(X.shape)
print(y.shape)
X

In [ ]:
NB_KFOLDS = 5

accuracies = []
f1_scores = []

for train_index, test_index in KFold(n_splits=NB_KFOLDS).split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    classifier.fit(X_train, y_train)
    y_test_pred = classifier.predict(X_test)
    accuracies.append(round(accuracy_score(y_test, y_test_pred),2))
    f1_scores.append(f1_score(y_test, y_test_pred, average="micro"))
    
    print(confusion_matrix(y_test, y_test_pred))
    
    print(classification_report(y_test, y_test_pred, target_names=EVENT_ID.keys()))

print(f"\n\nAccuracies accross {NB_KFOLDS} folds: {accuracies}")
print(f"Mean F1-score: {np.mean(f1_scores):0.2f}")